# LangChain

Como vimos anteriormente podemos utilizar estos Large Language Models entrenados para hacerles preguntas y recibir sus respuestas.

Es como si tuvieramos una funcion que recibe un texto como entrada y devuelve un texto como salida. Donde el texto de entrada puede ser cualquier pregunta y el texto de respuesta sera esta respuesta magica que es capaz de darnos un LLM. 

Esto parece simple y tonto pero es el sueño de cualquier programador. Es como si fuera posible hacer una busqueda en google y luego poder hacer web scrapping del mejor resultado pero de manera mucho mas sencilla.

LangChain es una API que nos permite realizar estas consultas a los LLM y poder parsear el resultado de forma mas simple. Veamos la diferencia entre consultar directamente a OpenAI como hicimos en la unidad anterior y hacerlo utilizando LangChain:

In [ ]:
!pip install openai --upgrade --quiet
!pip install langchain --upgrade --quiet

In [ ]:
import openai

# Cargar token desde google drive
from google.colab import drive
drive.mount('/content/drive')
with open('drive/MyDrive/Tokens/openai.txt','r') as f:
    token = f.read()
    openai.api_key = token

In [ ]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
from langchain.prompts import ChatPromptTemplate

template_string = """Traduce el siguiente texto \
que esta delimitado por comillas triples \
al siguiente estilo {estilo}. \
texto: ```{texto}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

message = prompt_template.format_messages(
                    estilo="rapero",
                    texto="Estimado Carlos, hoy no podre ir a tomar \
                    el te a tu casa porque estoy resfriado.")

response = chat(message)
print(response.content)

## Parsear la salida

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

### Genero el schema

In [ ]:
precio_schema = ResponseSchema(name="precio",
                             description="Cual ha sido el precio pagado?\
                             Si no se encuentra la informacion devolver -1.")
calificacion_schema = ResponseSchema(name="calificacion",
                                      description="Que calificacion se le asigno al restaurante?\
                                      Si no se encuentra la informacion devolver -1.")

response_schemas = [precio_schema, 
                    calificacion_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

### Consulto al LLM

In [ ]:
customer_review = """\
La comida fue espectacular, lo mejor la tortilla de patatas \
y la tarta de queso. El precio bien, en total costo 23 euros.
Le doy una calificacion de 8.3.
"""

review_template_2 = """\
Para el siguiente texto obtener la siguiente informacion:

precio: Cual ha sido el precio pagado?\
Si no se encuentra la informacion devolver -1.

calificacion: Que calificacion se le asigno al restaurante?\
Si no se encuentra la informacion devolver -1.

texto: {texto}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
response = chat(messages)
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)
print(output_dict)

## Memoria

Los modelos generalmente estan hechos de forma que no recuerdan lo que les has dicho anteriormente. Para solucionar esto podemos ir guardando todo lo que vamos "conversando" con el modelo y pasarselo como entrada junto con la nueva pregunta o interaccion que queremos tener con el modelo. Veamos como LangChain nos permite hacer esto de forma sencilla:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(openai_api_key=token, temperature=0.0, model="gpt-3.5-turbo")

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hola mi nombre es Hernan")

In [ ]:
conversation.predict(input="Cuanto es 20 + 22?")

In [ ]:
conversation.predict(input="Cual es mi nombre?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hola"}, 
                    {"output": "Como estas?"})

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory.save_context({"input": "Bien, descansando"}, 
                    {"output": "Que bueno"})

In [ ]:
memory.load_memory_variables({})

## Limitar la memoria

Si la conversacion es muy larga la memoria puede crecer demasiado, para limitar la cantidad de memoria que queremos que el modelo tenga LangChain nos ofrece diferentes formas de hacerlo:
- WindowMemory: guarda las k ultimas preguntas junto con sus respectivas respuestas
- TokenBufferMemory: guarda los ultimos max_token_limit tokens
- ConversationSummaryBufferMemory: guarda un resumen de max_token_limit tokens de lo conversado anteriormente

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

## Referencias:
Esta unidad es un resumen traducido con algunas modificaciones del [curso de Andrew Ng de Deeplearning.ai](https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/)

- https://www.deeplearning.ai/short-courses/langchain-chat-with-your-data/

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)